# Saves an sql file with create table commands for data stored in one or more csv files (basic)
Reads a series of csv files with the data to be imported into new tables (one table for each csv) and saves an sql file with the sql create tables commands<br>
__Usage__
1. Assign value to parameters 
1. Execute the tables_columns_2_rename function and save the results to file (one for tables and one for columns 2.
1. Open the tables file: in the first column is the name of the file without textension and a comma. Fill in after the comma the name of the table; the file name will be added as a comment to the table.
1. Open the columns file; similar process to the one performed with the table file.
1. Execute the function write_sql_commands
1. Open the file and, if desired, adjust the column types, comments, etc.
1. Execute the file with psql


In [1]:
import csv
import glob
import os
from os.path import join


## Functions

In [17]:
def tables_columns_2_rename(files: [str], to_screen: bool=False) -> [[str], [str]]:
    """
    Escribe el nombre de todas las tablas en files para ser renombradas como nombre de las tablas
    Extrae la cabecera de cada fichero csv y escribe en un fichero todas la cabeceras para que sean
    renombradas como las columnas de las tablas
    ----
    files: lista de paths csv a tratar
    to_screen: if True print intermediate results
    ---
    return [name_tables, name_columns]
    name_tables: nombres de los ficheros sin path
    name_columns: nombres de las columnas
    """
    name_tables = []
    name_columns = []

    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            name_tables.append(table_name)
            for ir, row in enumerate(rfi):
                if to_screen:
                    print(table_name)
                    print(row)
                    print('')
                for col1 in row:
                    if col1 not in name_columns:
                        name_columns.append(col1)
                break

    return name_tables, name_columns


def names_get(file: str) -> {str:str}:
    """
    file: path/file con los nombres (tablas o columnas según file) en los ficheros
        csv y los que se van a utilizar en las tablas
    ----
    return:
    dict en el que key es el nombre en el fichero csv y atrib es nombre a utilizar
    """
    with open(file, 'r', encoding='utf-8') as fi:
        rfi = csv.reader(fi, delimiter=',')
        names = [(row[0], row[1])  for row in rfi]
    return dict(names)
        

def sets_get():
    """
    valores de los parámetros para escribir en el fichero sql donde se crean las tablas
    """
    sets = \
"""
-- PostgreSQL database dump

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

"""    
    return sets


def write_sql_commands(dst: str, files: [str], table_names: {str:str}, column_names: {str:str},
                             schema: [str]='tmp', comment_prefix: str='',  to_screen: bool=False):
    """
    Escribe el fichero sql con los comandos create table 
    ----
    dst: path/fichero de resultados
    files: lista de paths csv a tratar
    table_names: dict {name in csv : name in table}
    column_names: dict {name in csv : name in table}
    schema: donde se crearán las tablas
    to_screen: if True print intermediate results
    ---
    return None
    """
    with open(dst, 'w', encoding='utf-8') as fo:
        fo.write(sets_get())
        for file in files:
            col_names = []
            with open(file, mode='r', encoding='utf-8') as fi:
                rfi = csv.reader(fi, delimiter=',')
                file_name, _ = os.path.basename(file).split('.')
                if file_name in table_names:
                    tab_name = table_names[file_name]
                else:
                    raise ValueError(f'No se localiza {file_name}')

                for ir, row in enumerate(rfi):
                    if to_screen:
                        print(table_name)
                        print(row)
                        print('')
                    for col1 in row:
                        if col1 in column_names:
                            col_names.append(column_names[col1])
                        else:
                            raise ValueError(f'No se localiza {col1}')
                            
                    sql_command = sql_command_get(schema, tab_name, file_name, col_names, row, comment_prefix)
                    
                    if to_screen:
                        print(table_names[file_name])
                    break
            
            fo.write(f'{sql_command}\n\n')
        
        return

    
def sql_command_get(schema: str, table_name: str, table_comment: str, 
                    col_names: [str], col_comments: [str]) -> str :
    """
    sets the sql commands for a table 
    """
    cols = [col1 + ' varchar' for col1 in col_names]
    cols = ',\n'.join(cols)
    if len(schema) > 0:
        schema = schema + '.'
    table_comment = table_comment.replace('_', ' ')
    comment = f"comment on table {schema}{table_name} is '{comment_prefix} {table_comment}';\n"
    
    comment_columns = [f"comment on columnn {schema}{table_name}.{coln} is '{colc}';" for coln, colc in zip(col_names, col_comments)]
    comment_columns = '\n'.join(comment_columns)
    
    command = f"""
create table if not exists {schema}{table_name} (
{cols}    
);

{comment}
{comment_columns}
    """
    return command

## Set parameter values

In [15]:
# directorio y máscara to get csv files
drt = r'E:\PPHH\chj_22_27\tablas'
mask = '*.csv'

# file_name_tables: nombre del fichero donde se grabarán el nombre d elas tablas
file_name_tables = join(drt, '01_table_names.txt')
# file_name_columns: nombre del fichero donde se grabarán el nombre de las columnas 
file_name_columns = join(drt, '02_colum_names.txt')

# fo_name: nombre del fichero donde se grabarán los comandos parara creación de las tablas
fo_name = join(drt, '03_create_tables.sql')

# schema: nombre del esquema de la DB donde se crearán las tablas
schema = 'tmp'


## Function 1. Write names of files and columns
After saving the 2 files (tables and columns), each file must be edited and the new names must be given manually

In [6]:
files = glob.glob(join(drt, mask))

tables, columns = tables_columns_2_rename(files)

with open(file_name_tables, 'w', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for t1 in tables:
        fo.write(f'{t1},\n')

with open(file_name_columns, 'w', newline='', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for c1 in columns:
        fo.write(f'{c1},\n')


## Function 2. write_sql_commands
After assigning the names of the tables and columns from the 2 files created in the previous cell, an sql file is created in which the tables are created

In [12]:
table_names = names_get(join(drt, '01_table_names_done.txt'))
column_names = names_get(join(drt, '02_colum_names_done.txt'))

write_sql_commands(fo_name, files, table_names, column_names, schema, comment_prefix='CHJ Plan Hidrológico 21-27')